In [37]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

print("=" * 80)
print("데이터 로드 및 결측치 제거")
print("=" * 80)

# ==========================================
# 1. 데이터 불러오기
# ==========================================

print("\n데이터 로딩 중...")
df_goemotions = pd.read_csv('data/clean_goemotions.csv')
df_reddit = pd.read_csv('data/clean_reddit.csv')

# 결측치(빈 텍스트) 제거 (저장/로드 과정에서 생길 수 있는 NaN 제거)
df_goemotions.dropna(subset=['text'], inplace=True)
df_reddit.dropna(subset=['body'], inplace=True)

print(f"GoEmotions 데이터: {df_goemotions.shape}")
print(f"Reddit 데이터: {df_reddit.shape}")

데이터 로드 및 결측치 제거

데이터 로딩 중...
GoEmotions 데이터: (57160, 30)
Reddit 데이터: (36941, 2)


In [38]:
print("=" * 80)
print("Tokenizer 실행 (텍스트 -> 정수 인덱스)")
print("=" * 80)

# ==========================================
# 2. Tokenizer 실행 (텍스트 -> 정수 인덱스)
# ==========================================
# 두 데이터를 합쳐서 전체 단어 사전을 만듭니다 (Reddit의 신조어도 포함하기 위해)
all_texts = list(df_goemotions['text'].astype(str)) + list(df_reddit['body'].astype(str))

# Tokenizer 객체 생성 (<OOV>는 사전에 없는 단어 처리용)
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(all_texts)

# 단어 집합 크기 (Vocab Size) + 1 (패딩용 0 때문에 +1)
vocab_size = len(tokenizer.word_index) + 1
print(f"\n전체 단어 집합 크기: {vocab_size:,}개")

# 텍스트를 정수 시퀀스로 변환
goemotions_sequences = tokenizer.texts_to_sequences(df_goemotions['text'].astype(str))
reddit_sequences = tokenizer.texts_to_sequences(df_reddit['body'].astype(str))

# 변환 예시 확인
print(f"\n[변환 예시]")
print(f"원문: {df_goemotions['text'].iloc[0]}")
print(f"정수 인덱스: {goemotions_sequences[0]}")

Tokenizer 실행 (텍스트 -> 정수 인덱스)

전체 단어 집합 크기: 41,690개

[변환 예시]
원문: that game hurt
정수 인덱스: [9, 168, 659]


In [39]:
print("=" * 80)
print("패딩 (Padding) - 길이 맞추기")
print("=" * 80)

# ==========================================
# 3. 패딩 (Padding) - 길이 맞추기
# ==========================================
# 문장 길이를 맞춰줍니다. (40으로 설정)
max_length = 40
padding_type = 'post'   # 문장 뒤에 0을 채움 (0, 0, 0...)
trunc_type = 'post'     # 문장이 길면 뒤를 자름

goemotions_padded = pad_sequences(goemotions_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
reddit_padded = pad_sequences(reddit_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(f"\n[패딩 결과]")
print(f"GoEmotions 입력 데이터 Shape: {goemotions_padded.shape}")
print(f"Reddit 입력 데이터 Shape: {reddit_padded.shape}")
print(f"\n패딩된 데이터 예시: {goemotions_padded[0]}")

패딩 (Padding) - 길이 맞추기

[패딩 결과]
GoEmotions 입력 데이터 Shape: (57160, 40)
Reddit 입력 데이터 Shape: (36941, 40)

패딩된 데이터 예시: [  9 168 659   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]


In [40]:
print("=" * 80)
print("임베딩(Word2Vec) 변환")
print("=" * 80)

# ==========================================
# 4. 임베딩(Word2Vec) 변환
# ==========================================

# 임베딩 설정
embedding_dim = 64  # 하나의 단어를 64차원 벡터로 표현

# Keras의 Embedding 레이어 객체 생성
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

# 샘플 데이터 하나만 통과시켜보기 (첫 번째 문장)
sample_input = goemotions_padded[0]  # (40,) 정수 배열
sample_output = embedding_layer(sample_input) # 임베딩 층 통과

print(f"\n--- [변환 결과 확인] ---")
print(f"입력 문장(정수 인덱스): {sample_input[:10]} ... (길이: {len(sample_input)})")
print(f"임베딩 후 벡터 형태(Shape): {sample_output.shape} (문장길이 40 x 벡터차원 64)")
print(f"첫 번째 단어의 벡터값(일부): {sample_output[0][:5].numpy()} ...")
print("\n------------------------")
print("※ 현재 벡터값은 랜덤 초기화 상태입니다. 모델 학습(fit)을 돌리면 의미 있는 값으로 업데이트됩니다.")

임베딩(Word2Vec) 변환

--- [변환 결과 확인] ---
입력 문장(정수 인덱스): [  9 168 659   0   0   0   0   0   0   0] ... (길이: 40)
임베딩 후 벡터 형태(Shape): (40, 64) (문장길이 40 x 벡터차원 64)
첫 번째 단어의 벡터값(일부): [-0.03509829  0.00182395 -0.03594821 -0.0102967  -0.03926591] ...

------------------------
※ 현재 벡터값은 랜덤 초기화 상태입니다. 모델 학습(fit)을 돌리면 의미 있는 값으로 업데이트됩니다.


In [41]:
print("=" * 80)
print("토크나이저 저장 (나중에 사용하기 위해)")
print("=" * 80)

# ==========================================
# 5. 토크나이저 저장 (나중에 사용하기 위해)
# ==========================================
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("\nTokenizer 저장 완료")

토크나이저 저장 (나중에 사용하기 위해)

Tokenizer 저장 완료
